In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('msc_data_cleaned_new1.csv')

In [2]:
def clean_age(x):
    if '51-60' in x or '60+' in x:
        return '51-60'
    if '41-45' in x or '46-50' in x:
        return '41-50'
    if '31-35' in x or '36-40' in x:
        return '31-40'
    if '21-25' in x or '26-30' in x:
        return '21-30'
    return 'Under 20'

dataset['age_cat'] = dataset['age_cat'].apply(clean_age)

In [3]:
dataset = dataset.dropna(subset=['device_type'])

In [4]:
def clean_deviceType(x):
    if 'BASIC' in x:
        return 'BASIC'
    if 'SMART' in x:
        return 'SMART'
    if 'FEATURE' in x:
        return 'FEATURE'

dataset['device_type'] = dataset['device_type'].apply(clean_deviceType)

In [5]:
sin= 2078.0
eng = 2002.0
# tam = 2079.0
def clean_language(x):
    if sin in [x]:
        return 'Sinhala'
    if eng in [x]:
        return 'English'
    return 'Tamil'

dataset['language_id'] = dataset['language_id'].apply(clean_language)

In [6]:
def clean_connectionAge(x):
    if 'more than 5 years' in x:
        return 'more than 5 years'
    if '3-5 years' in x:
        return '3-5 years'
    if '2-3 years' in x or '1-2 years' in x:
        return '1-3 years'
    return 'less than 1'

dataset['con_age'] = dataset['con_age'].apply(clean_connectionAge)

In [7]:
from sklearn.preprocessing import LabelEncoder
# labelEncoder = LabelEncoder()
dataTransform = dataset.copy()

In [8]:
le_language = LabelEncoder()
dataTransform['language_id'] = le_language.fit_transform(dataset['language_id'])
le_age = LabelEncoder()
dataTransform['age_cat'] = le_age.fit_transform(dataset['age_cat'])
le_conAge = LabelEncoder()
dataTransform['con_age'] = le_conAge.fit_transform(dataset['con_age'])
le_device = LabelEncoder()
dataTransform['device_type'] = le_device.fit_transform(dataset['device_type'])
le_package = LabelEncoder()
dataTransform['package'] = le_package.fit_transform(dataset['package'])

In [9]:
dataTransform

,package,gender,age_cat,con_age,language_id,total_voice_usage_min,total_data_usage_mb,device_type,total_revenue,vas_revenue
0,2,0,3,2,0,76.25,12177.65,2,1319.92,226.67
2,1,0,1,0,1,143.83,7318.22,2,1357.65,143.43
3,2,1,2,2,0,128.55,0.21,2,277.25,154.28
4,1,1,2,0,0,15.27,20411.27,2,1797.99,190.03
6,1,0,3,2,0,825.47,0.00,1,1230.00,127.50
...,...,...,...,...,...,...,...,...,...,...
26274,0,0,3,3,0,320.47,2587.44,2,1017.33,179.72
26276,1,1,3,3,1,223.58,0.00,1,365.07,158.15
26277,1,1,3,3,1,616.03,0.00,0,884.65,115.15
26278,1,0,3,1,0,474.97,494.23,2,661.97,180.52


In [10]:
dataTransform.shape

(21781, 10)

In [11]:
dataTransform1 = dataTransform.copy()
dtf = dataTransform1['package']
# Min-Max Normalization
df = dataTransform1.drop('package', axis=1)
dataTransform = dataTransform.drop('package', axis=1)
df_norm = (df-df.min())/(df.max()-df.min())

df_norm.head()

,gender,age_cat,con_age,language_id,total_voice_usage_min,total_data_usage_mb,device_type,total_revenue,vas_revenue
0,0.0,0.75,0.666667,0.0,0.008460,0.098454,1.0,0.019094,0.003322
2,0.0,0.25,0.000000,0.5,0.015958,0.059166,1.0,0.019646,0.002089
3,1.0,0.50,0.666667,0.0,0.014263,0.000002,1.0,0.003819,0.002250
4,1.0,0.50,0.000000,0.0,0.001694,0.165021,1.0,0.026097,0.002780
6,0.0,0.75,0.666667,0.0,0.091588,0.000000,0.5,0.017776,0.001853


In [12]:
def outliers(df,feature):
    Q1 = df[feature].quantile(0.05)
    Q3 = df[feature].quantile(0.95)
    IQR = Q3 - Q1
    
    lowerBound = Q1 - 1.5*IQR
    uperBound  = Q3 + 1.5*IQR
    
    ls = df.index[(df[feature]<lowerBound) | (df[feature]>uperBound)]
    return ls

In [13]:
index_list = []
for f in ['total_voice_usage_min','total_data_usage_mb','total_revenue']:
    index_list.extend(outliers(df_norm,f))

In [14]:
def remove(df,ls):
    ls = sorted(set(ls))
    df =df.drop(ls)
    return df

In [15]:
cleand_df = remove(df_norm,index_list)

In [16]:
cleand_df.shape

(21584, 9)

In [17]:
result = pd.concat([cleand_df, dtf], axis=1, join='inner')
display(result)

,gender,age_cat,con_age,language_id,total_voice_usage_min,total_data_usage_mb,device_type,total_revenue,vas_revenue,package
0,0.0,0.75,0.666667,0.0,0.008460,9.845378e-02,1.0,0.019094,0.003322,2
2,0.0,0.25,0.000000,0.5,0.015958,5.916630e-02,1.0,0.019646,0.002089,1
3,1.0,0.50,0.666667,0.0,0.014263,1.697807e-06,1.0,0.003819,0.002250,2
4,1.0,0.50,0.000000,0.0,0.001694,1.650209e-01,1.0,0.026097,0.002780,1
6,0.0,0.75,0.666667,0.0,0.091588,0.000000e+00,0.5,0.017776,0.001853,1
...,...,...,...,...,...,...,...,...,...,...
26274,0.0,0.75,1.000000,0.0,0.035557,2.091892e-02,1.0,0.014661,0.002627,0
26276,1.0,0.75,1.000000,0.5,0.024807,0.000000e+00,0.5,0.005106,0.002307,1
26277,1.0,0.75,1.000000,0.5,0.068350,0.000000e+00,0.0,0.012717,0.001670,1
26278,0.0,0.75,0.333333,0.0,0.052699,3.995747e-03,1.0,0.009455,0.002639,1


In [18]:
y = np.array(result.iloc[:,9:])
X = np.array(result.iloc[:,0:9])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


In [19]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
X1, y1=smote.fit_resample(X_train, y_train)


In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores_cart = cross_val_score(DecisionTreeClassifier(),X,y,cv=10)
scores_cart

array([0.77165354, 0.77026401, 0.76748495, 0.77443261, 0.75486562,
       0.76274328, 0.77757183, 0.77015755, 0.77386469, 0.76042632])

In [21]:
scores_cart.mean()

0.7683464395222962

In [ ]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)

model.fit(X_train,y_train)
scores_bag_cart = cross_val_score(model, X, y, cv=10)
scores_bag_cart
model.score(X_test,y_test)
y_pred=model.predict(X_test)

C:\Users\sandunic\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sandunic\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("Bagged CART Accuracy: %.3f" % (scores_bag_cart.mean() * 100))

In [ ]:
import pickle
# pickle.dump(rf, open('randomForest.pkl', 'wb'))
data = {"model": model, "le_age": le_age, "le_device":le_device, "le_language":le_language, "le_conAge":le_conAge, "le_package":le_package}
with open('baggedcart.pkl', 'wb') as file:
    pickle.dump(data, file)

In [ ]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

In [ ]:
confusion_matrix

In [ ]:
from yellowbrick.classifier import ROCAUC

In [ ]:
from yellowbrick.classifier import ROCAUC

def plot_ROC_curve(model, X_train, X_test, y_train, y_test):

    # Creating visualization with the readable labels
    visualizer = ROCAUC(model, encoder={0: 'B2W', 
                                        1: 'CRBT_SRV', 
                                        2: 'MQSM'})
                                        
    # Fitting to the training data first then scoring with the test data                                    
    visualizer.fit(X_train, y_train)
    visualizer.score(X_test, y_test)
    visualizer.show()
    
    return visualizer

In [ ]:
plot_ROC_curve(model, X_train, X_test, y_train, y_test)